In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
def reshape_df(df):
    #idは意味を持たないため削除
    df.drop('id',axis=1,inplace=True)

    #print(train_df["StandardHours"].value_counts())
    #80    1200
    #Name: StandardHours, dtype: int64
    df.drop('StandardHours',axis=1,inplace=True)

    #print(train_df["Over18"].value_counts())
    #Y    1200
    #Name: Over18, dtype: int64
    df.drop('Over18',axis=1,inplace=True)

    #適当に
    '''
    df.drop('EmployeeNumber',axis=1,inplace=True)
    df.drop('JobLevel',axis=1,inplace=True)
    df.drop('MonthlyIncome',axis=1,inplace=True)
    df.drop('YearsAtCompany',axis=1,inplace=True)
    df.drop('YearsSinceLastPromotion',axis=1,inplace=True)
    df.drop('WorkLifeBalance',axis=1,inplace=True)
    df.drop('StandardHours',axis=1,inplace=True)
    df.drop('NumCompaniesWorked',axis=1,inplace=True)
    '''

    #https://deepage.net/features/pandas-replace.html
    df=df.replace({
        #BusinessTravel
        'Non-Travel':0,
        'Travel_Rarely':1,
        'Travel_Frequently':2,
        #Gender
        'Male':0,
        'Female':1,
        #OverTime
        'No':0,
        'Yes':1,
    })

    #https://note.nkmk.me/python-pandas-get-dummies/
    df=pd.get_dummies(df, columns=["Department", "EducationField","JobRole","MaritalStatus"],sparse=True)
    return df

import scipy.stats
def preprocess(df):
    df=reshape_df(df)
    return scipy.stats.zscore(df.values,ddof=1)

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

primary_df=pd.read_csv('/content/drive/MyDrive/signate/従業員の離職予測/train.csv')
ans_df=primary_df.loc[:,'Attrition']
primary_df.drop('Attrition',axis=1,inplace=True)

x_train,x_test,y_train,y_test=train_test_split(primary_df,ans_df, test_size=0.1)

x_train=preprocess(x_train)
x_test=preprocess(x_test)

In [38]:
import lightgbm as lgb
lgb_train = lgb.Dataset(x_train, y_train)
lgb_test = lgb.Dataset(x_test, y_test, reference=lgb_train)
# LightGBM のハイパーパラメータ
params = {
    # 二値分類問題
    'objective': 'binary',
    # AUC の最大化を目指す
    #'metric': 'binary_logloss',
    'metric': 'acc',
    # Fatal の場合出力
    'verbosity': -1,
}
# 上記のパラメータでモデルを学習する
model = lgb.train(
    params = params,
    train_set = lgb_train,
    valid_sets = [lgb_train, lgb_test],
    num_boost_round=25000,
                 )
# テストデータを予測する
y_pred = model.predict(x_test, num_iteration=model.best_iteration)

In [33]:
y_pred

array([1.00105053e-11, 8.17716964e-01, 6.18872134e-04, 1.70512276e-05,
       1.76304482e-08, 4.14457306e-06, 2.41615258e-05, 3.65266841e-09,
       5.42492558e-08, 2.70463549e-06, 2.18817353e-05, 1.55228066e-01,
       5.18045038e-10, 2.45823999e-07, 1.00925159e-05, 1.92195920e-07,
       2.52216508e-04, 8.35480849e-05, 3.06146066e-03, 7.23655381e-10,
       2.95431543e-11, 3.72633850e-08, 2.88717411e-04, 2.99438294e-03,
       1.29105864e-05, 9.02769553e-04, 1.69378600e-11, 2.29133507e-04,
       9.85069849e-01, 3.19012470e-07, 8.86258559e-11, 1.62952165e-08,
       2.31330819e-07, 1.17419633e-07, 1.77714861e-06, 4.20717255e-09,
       2.79225058e-06, 1.59426515e-06, 4.96492042e-03, 1.59603877e-05,
       3.51201610e-05, 7.47101715e-09, 1.03981227e-07, 1.28644295e-05,
       4.32817466e-02, 8.60329245e-02, 1.21979047e-04, 3.16664947e-09,
       8.76601279e-08, 5.03988432e-10, 4.03993242e-03, 6.12714942e-02,
       6.55311565e-03, 3.15594988e-04, 3.88101906e-09, 7.40934638e-05,
      

In [39]:
y_pred[y_pred<0.5]=0
y_pred[y_pred>=0.5]=1
y_test

355     1
966     1
243     0
175     0
300     0
       ..
809     0
538     0
913     0
1088    0
351     0
Name: Attrition, Length: 120, dtype: int64

In [40]:
(y_pred==y_test).sum()/len(y_pred)

0.9

In [41]:
import numpy as np
test_df=pd.read_csv('/content/drive/MyDrive/signate/従業員の離職予測/test.csv')
x_test=preprocess(test_df)
pred=model.predict(x_test, num_iteration=model.best_iteration)
pred[pred<0.5]=0
pred[pred>=0.5]=1

test_df=pd.read_csv('/content/drive/MyDrive/signate/従業員の離職予測/test.csv')
data=np.array([test_df["id"].values,pred],dtype='int16')
data=pd.DataFrame(data.T)
data.to_csv('/content/drive/MyDrive/signate/従業員の離職予測/out25000acc.csv',index=False,header=False)